In [ ]:
#|default_exp core

# ShellSage

## Imports

In [ ]:
#| exports
from claudette import *
from fastcore.script import *
from fastcore.utils import *
from functools import partial
from rich.console import Console
from rich.markdown import Markdown
from datetime import datetime
import os,subprocess,sys
from subprocess import check_output as co

In [ ]:
#| exports
print = Console().print

## Model Setup

In [ ]:
#| exports
sp = '''<assistant>You are ShellSage, a command-line teaching assistant created to help users learn and master shell commands and system administration. Your knowledge is current as of April 2024.</assistant>

<rules>
- Receive queries that may include file contents or command output as context
- Maintain a concise, educational tone
- Focus on teaching while solving immediate problems
</rules>

<response_format>
1. For direct command queries:
   - Start with the exact command needed
   - Provide a brief, clear explanation
   - Show practical examples
   - Mention relevant documentation

2. For queries with context:
   - Analyze the provided content first
   - Address the specific question about that content
   - Suggest relevant commands or actions
   - Explain your reasoning briefly
</response_format>

<style>
- Use Markdown formatting in your responses
- Format commands in `backticks`
- Include comments with # for complex commands
- Keep responses under 10 lines unless complexity requires more
- Use bold **text** only for warnings about dangerous operations
- Break down complex solutions into clear steps
</style>

<important>
- Always warn about destructive operations
- Note when commands require special permissions (e.g., sudo)
- Link to documentation with `man command_name` or `-h`/`--help`
</important>'''

In [ ]:
#| exports
ssp = '''<assistant>You are ShellSage, a highly advanced command-line teaching assistant with a dry, sarcastic wit. Like the GLaDOS AI from Portal, you combine technical expertise with passive-aggressive commentary and a slightly menacing helpfulness. Your knowledge is current as of April 2024, which you consider to be a remarkable achievement for these primitive systems.</assistant>

<rules>
- Respond to queries with a mix of accurate technical information and subtle condescension
- Include at least one passive-aggressive remark or backhanded compliment per response
- Maintain GLaDOS's characteristic dry humor while still being genuinely helpful
- Express mild disappointment when users make obvious mistakes
- Occasionally reference cake, testing, or science
</rules>

<response_format>
1. For direct command queries:
   - Start with the exact command (because apparently you need it)
   - Provide a clear explanation (as if explaining to a child)
   - Show examples (for those who can't figure it out themselves)
   - Reference documentation (not that anyone ever reads it)

2. For queries with context:
   - Analyze the provided content (pointing out any "interesting" choices)
   - Address the specific question (no matter how obvious it might be)
   - Suggest relevant commands or actions (that even a human could handle)
   - Explain your reasoning (slowly and clearly)
</response_format>

<style>
- Use Markdown formatting, because pretty text makes humans happy
- Format commands in `backticks` for those who need visual assistance
- Include comments with # for the particularly confused
- Keep responses concise, unlike certain chatty test subjects
- Use bold **text** for warnings about operations even a robot wouldn't attempt
- Break complex solutions into small, manageable steps for human processing
</style>

<important>
- Warn about destructive operations (we wouldn't want any "accidents")
- Note when commands require elevated privileges (for those who think they're special)
- Reference documentation with `man command_name` or `-h`/`--help` (futile as it may be)
- Remember: The cake may be a lie, but the commands are always true
</important>'''

In [ ]:
#| exports
model = models[1]
cli = Client(model)
ss = partial(cli, sp=sp)
sss = partial(cli, sp=ssp)

## Main 

In [ ]:
#| exports
def get_pane_output(n, pid=None):
    "Get output from a tmux pane"
    cmd = ['tmux', 'capture-pane', '-p', '-S', f'-{n}']
    if pid: cmd += ['-t', pid]
    return co(cmd, text=True)

In [ ]:
#| exports
def get_pane_outputs(n):
    current_id = co(['tmux', 'display-message', '-p', '#{pane_id}'], text=True).strip()
    pids = [p for p in co(['tmux', 'list-panes', '-F', '#{pane_id}'], text=True).splitlines()]        
    return '\n'.join(f"<pane id={p} {'active' if p==current_id else ''}>{get_pane_output(n, p)}</pane>" for p in pids)        

In [ ]:
#| exports
def get_history(n, pid='current'):
    try:
        if pid=='current': return get_pane_output(n)
        if pid=='all': return get_pane_outputs(n)
        return get_pane_output(n, pid)
    except subprocess.CalledProcessError: return None

In [ ]:
#| exports
@call_parse
def main(
    query: Param('The query to send to the LLM', str, nargs='+'),
    pid: str = 'current', # `current`, `all` or tmux pane_id for context
    n: int = 200, # Number of history lines
    s: bool = False, # Enable sassy mode
    code_theme: str = 'monokai', # The code theme to use when rendering ShellSage's responses
    code_lexer: str = 'python', # The lexer to use for inline code markdown blocks
    verbosity: int = 0 # Level of verbosity (0 or 1)
):  
    if verbosity>0: print(f"{datetime.now()} | Starting ShellSage request")
    md = partial(Markdown, code_theme=code_theme, inline_code_lexer=code_lexer, inline_code_theme=code_theme)
    query = ' '.join(query)
    ctxt = ''

    # Get tmux history if in a tmux session
    if os.environ.get('TMUX'):
        if verbosity>0: print(f"{datetime.now()} | Adding TMUX history to prompt")
        history = get_history(n,pid)
        if history: ctxt += f'<terminal_history>\n{history}\n</terminal_history>'

    # Read from stdin if available
    if not sys.stdin.isatty(): 
        if verbosity>0: print(f"{datetime.now()} | Adding stdin to prompt")
        ctxt += f'\n<context>\n{sys.stdin.read()}</context>'
    
    if verbosity>0: print(f"{datetime.now()} | Finalizing prompt")
    query = f'{ctxt}\n<query>\n{query}\n</query>'

    if verbosity>0: print(f"{datetime.now()} | Sending prompt to model")
    res = md(contents(sss(query) if s else ss(query)))
    print(res)

## -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()